In [1]:
# Load dependencies
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.1e}'.format
import sys
sys.path.insert(0,'../../../../statistics_helper/')
from excel_utils import *

# Estimating the biomass of poultry
To estimate the biomass of poultry, we rely on data on global stocks of chickens, ducks, and turkeys from the Food and Agriculture Organization database FAOStat. We downloaded data from the domain Production/Live animals.
We combined data on the total stocks of each animal with estimates of the mean mass of each type of animal species (in kg) from [Dong et al.](http://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_10_Ch10_Livestock.pdf), Annex 10A.2, Tables 10A-4 to 10A-9.

Here are samples of the data:

In [2]:
# Load bird data
bird = pd.read_csv('FAOSTAT_data_bird.csv')
bird.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QA,Live Animals,5100,Africa,5112,Stocks,1057,Chickens,2014,2014,1000 Head,1809059,A,"Aggregate, may include official, semi-official..."
1,QA,Live Animals,5100,Africa,5112,Stocks,1068,Ducks,2014,2014,1000 Head,28539,A,"Aggregate, may include official, semi-official..."
2,QA,Live Animals,5100,Africa,5112,Stocks,1072,Geese and guinea fowls,2014,2014,1000 Head,25296,A,"Aggregate, may include official, semi-official..."
3,QA,Live Animals,5100,Africa,5112,Stocks,1083,"Pigeons, other birds",2014,2014,1000 Head,13384,A,"Aggregate, may include official, semi-official..."
4,QA,Live Animals,5100,Africa,5112,Stocks,1079,Turkeys,2014,2014,1000 Head,23658,A,"Aggregate, may include official, semi-official..."


In [3]:
# Load body mass data
body_mass = pd.read_csv('ipcc_animal_weight.csv')
body_mass.set_index('IPCC Area',inplace=True)
body_mass.head()

,Cattle - dairy,Cattle - non-dairy,Buffaloes,Swine - market,Swine - breeding,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys,Sheep,Goats,Horses,Asses,Mules,Camels,Llamas
IPCC Area,,,,,,,,,,,,,,,,
Indian Subcontinent,275,110,295,28,28,0.9,1.8,2.7,6.8,28.0,30.0,238,130,130,217,217
Eastern Europe,550,391,380,50,180,0.9,1.8,2.7,6.8,48.5,38.5,377,130,130,217,217
Africa,275,173,380,28,28,0.9,1.8,2.7,6.8,28.0,30.0,238,130,130,217,217
Oceania,500,330,380,45,180,0.9,1.8,2.7,6.8,48.5,38.5,377,130,130,217,217
Western Europe,600,420,380,50,198,0.9,1.8,2.7,6.8,48.5,38.5,377,130,130,217,217


We pivot the stocks DataFrame to have a view of each kind of animal at each region:

In [4]:

# Replace NaN with zeros
bird.fillna(value=0,inplace=True)

bird_pivot = pd.pivot(bird.Area,bird.Item, bird.Value).astype(float)

# Replace NaN with zeros
bird_pivot.fillna(value=0,inplace=True)


bird_pivot

Item,Chickens,Ducks,Geese and guinea fowls,"Pigeons, other birds",Turkeys
Area,,,,,
Africa,1809059.0,28539.0,25296.0,13384.0,23658.0
Americas,5436151.0,27594.0,781.0,0.0,312477.0
Asia,11923472.0,986528.0,281715.0,16091.0,14575.0
Eastern Europe,1005785.0,54155.0,17095.0,0.0,37218.0
Northern America,2149990.0,9251.0,350.0,0.0,243200.0
Oceania,126014.0,1428.0,82.0,0.0,1377.0
Southern Asia,2841669.0,76462.0,1000.0,0.0,2000.0
Western Europe,452917.0,29768.0,865.0,1800.0,33255.0


There is a difference between the body mass of a egg laying chicken to a non-egg laying chicken. We thus count seperately the egg laying chicken from the non-egg laying chicken. Data about the amount of egg laying chicken comes from the FAOStat domain Production - Livestock Primary.

In [5]:
# Load data about egg laying chicken
egg = pd.read_csv('FAOSTAT_data_eggs.csv')

# Set the index of the DataFrame to be the region so we can compare with the stocks data
egg.set_index('Area',inplace=True)

# Add a category of egg laying chicken
bird_pivot['Chicken - Layers'] = egg.Value

# Set the amount of non-egg laying chicken to be the total number minus the egg laying chicken
bird_pivot['Chickens'] -= egg.Value

# Rename the Chicken column name to Chicken - Broileers
bird_pivot.rename(columns={'Chickens': 'Chicken - Broilers'},inplace=True)

# Use only data for chicken, ducks and turkeys
birds = ['Chicken - Broilers','Chicken - Layers','Ducks','Turkeys']
bird_pivot_filt = bird_pivot[birds]
body_mass_filt = body_mass[birds]

bird_pivot_filt

Item,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys
Area,,,,
Africa,1303856.0,505203,28539.0,23658.0
Americas,4286851.0,1149300,27594.0,312477.0
Asia,7388298.0,4535174,986528.0,14575.0
Eastern Europe,581415.0,424370,54155.0,37218.0
Northern America,1751965.0,398025,9251.0,243200.0
Oceania,103680.0,22334,1428.0,1377.0
Southern Asia,2181552.0,660117,76462.0,2000.0
Western Europe,284558.0,168359,29768.0,33255.0


Data on the mass of animals is divided into different regions than the FAOStat data so we need preprocess the stocks DataFrame and merge it with the body mass data:

In [6]:
# Convert units
bird_pivot_filt *= 1e3

# Calculate the total number of animals in Latin America by subtracting values for Northern America from the total
# values for the Americas
bird_pivot_filt.loc['Americas'] -= bird_pivot_filt.loc['Northern America']

# Change name of Americas to Latin America
bird_pivot_filt.rename(index={'Americas': 'Latin America'},inplace=True)

# Calculate the total number of animals in Asia without the Indian Subcontinent by subtracting values for the Southern Asia 
# from the total values for the Asia
bird_pivot_filt.loc['Asia'] -= bird_pivot_filt.loc['Southern Asia']

# Change name of Southern Asia to Indian Subcontinent
bird_pivot_filt.rename(index={'Southern Asia': 'Indian Subcontinent'},inplace=True)

bird_pivot_filt

/home/yinon/.local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/yinon/.local/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:13

Item,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys
Area,,,,
Africa,1.303856e+09,5.052030e+08,28539000.0,23658000.0
Latin America,2.534886e+09,7.512750e+08,18343000.0,69277000.0
Asia,5.206746e+09,3.875057e+09,910066000.0,12575000.0
Eastern Europe,5.814150e+08,4.243700e+08,54155000.0,37218000.0
Northern America,1.751965e+09,3.980250e+08,9251000.0,243200000.0
Oceania,1.036800e+08,2.233400e+07,1428000.0,1377000.0
Indian Subcontinent,2.181552e+09,6.601170e+08,76462000.0,2000000.0
Western Europe,2.845580e+08,1.683590e+08,29768000.0,33255000.0


We now multiply the stocks of each animal type and for each region by the characteristic body weight of each animal:

In [7]:
wet_bird_biomass = (body_mass_filt*bird_pivot_filt)
wet_bird_biomass

,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys
Africa,1.173470e+09,9.093654e+08,7.705530e+07,1.608744e+08
Asia,4.686071e+09,6.975103e+09,2.457178e+09,8.551000e+07
Eastern Europe,5.232735e+08,7.638660e+08,1.462185e+08,2.530824e+08
Indian Subcontinent,1.963397e+09,1.188211e+09,2.064474e+08,1.360000e+07
Latin America,2.281397e+09,1.352295e+09,4.952610e+07,4.710836e+08
Middle east,NaN,NaN,NaN,NaN
Northern America,1.576768e+09,7.164450e+08,2.497770e+07,1.653760e+09
Oceania,9.331200e+07,4.020120e+07,3.855600e+06,9.363600e+06
Western Europe,2.561022e+08,3.030462e+08,8.037360e+07,2.261340e+08


We sum over all regions and convert units from kg wet weight to Gt C carbon by assuming carbon is ≈15% of the wet weight (30% dry weight of wet weight and carbon is 50% of dry weight).

In [8]:
pd.options.display.float_format = '{:,.3f}'.format

# conversion factor from kg wet weight to Gt C
kg_to_gt_c = 1000*0.15/1e15
total_biomass = wet_bird_biomass.sum()*kg_to_gt_c
total_biomass

Chicken - Broilers    0.001883
Chicken - Layers      0.001837
Ducks                 0.000457
Turkeys               0.000431
dtype: float64

In [9]:
best_estimate = total_biomass.sum()
print('Our estimate for the total biomass of poultry is ≈%.3f Gt C' % best_estimate)

Our estimate for the total biomass of poultry is ≈0.005 Gt C


In [ ]:
update_MS_data(row='Biomass of poultry',values= bird_biomass.sum(),path='../../../../results.xlsx')